In [1]:
import os, sys
sys.path.append('/Users/dimeji/Documents/Projects/geo_earth_temp/getemp')

# 1.  Import python dependencies

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio.plot
import rasterio

# 2. Location

For this tutorial, we’ll use the NIR and Red bands from a Landsat-8 scene above part of the central valley and the Sierra Nevada in California. We’ll be using Level 1TP datasets, orthorectified, map-projected images containing radiometrically calibrated data.

# 3. Bands needed from land surface temperature computation (Split window)
- Red: Band 4
- Near-Infrared (NIR): Band 5
- Thermal infrared 1: Band 10
- Thermal infrared 2: Band 11

Here, I have used `rasterio` to load the images/bands needed.

In [3]:
url = 'http://landsat-pds.s3.amazonaws.com/c1/L8/042/034/LC08_L1TP_042034_20170616_20170629_01_T1/'
#url = 'https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/042/034/LC08_L1TP_042034_20170616_20170629_01_T1/'
redband = 'LC08_L1TP_042034_20170616_20170629_01_T1_B{}.TIF'.format(4) # L1TP_216074_20160531_20180528_01_T1_B4.TIF'
nirband = 'LC08_L1TP_042034_20170616_20170629_01_T1_B{}.TIF'.format(5)
tempband10 = 'LC08_L1TP_042034_20170616_20170629_01_T1_B{}.TIF'.format(10)
tempband11 = 'LC08_L1TP_042034_20170616_20170629_01_T1_B{}.TIF'.format(11)

In [4]:
#with rasterio.open(url+redband) as src:
#    redImage = src.read(1).astype('f4')
#
#with rasterio.open(url+nirband) as src:
#    nirImage = src.read(1).astype('f4')
#    
#with rasterio.open(url+tempband10) as src:
#    tempImage10 = src.read(1).astype('f4')
#
#with rasterio.open(url+tempband11) as src:
#    tempImage11 = src.read(1).astype('f4')

# 8. Compute land surface temperature

In [5]:
import numpy as np

In [16]:
class KeywordArgumentError(Exception):
    pass


def assert_all_keywords_are_provided(keywords=None, **kwargs):
    for keyword in keywords:
        if keyword not in kwargs:
            message = f"Keyword argument {keyword} must be providedfor this computation "
            raise KeywordArgumentError(message)

In [17]:
class SplitWindowParentLST:

    # A comparison of all the methods can be found here:
    # https://link.springer.com/article/10.1007/s40808-020-01007-1/tables/3

    def __init__(self):
        self.max_earth_temp = 273.15 + 56.7

    def __call__(self, **kwargs) -> np.ndarray:
        lst = self._compute_lst(**kwargs)
        lst[lst > self.max_earth_temp] = np.nan
        return lst

    def _compute_lst(self, **kwargs):
        raise NotImplementedError("Concrete method yet to be implemented")
        
class SplitWindowJiminezMunozLST(SplitWindowParentLST):
    cwv = 0.013

    """
    Method reference:

    Jiménez-Muñoz, Juan-Carlos, and José A. Sobrino. "Split-window coefficients for land surface
    temperature retrieval from low-resolution thermal infrared sensors." IEEE geoscience and
    remote sensing letters 5.4 (2008): 806-809.

    """

    def _compute_lst(self, **kwargs) -> np.ndarray:
        """Computes the LST

        kwargs:

        **emissivity_10 (np.ndarray): Emissivity image obtained for band 10
        **emissivity_11 (np.ndarray): Emissivity image obtained for band 11
        **brightness_temperature_10 (np.ndarray): Brightness temperature image obtained for band 10
        **brightness_temperature_11 (np.ndarray): Brightness temperature image obtained for band 11
        **mask (np.ndarray[bool]): Mask image. Output will have NaN value where mask is True.

        Returns:
            np.ndarray: Land surface temperature image
        """

        assert_all_keywords_are_provided(
            [
                "emissivity_10",
                "emissivity_11",
                "brightness_temperature_10",
                "brightness_temperature_11",
                "mask",
            ],
            **kwargs
        )

        tb_10 = kwargs["brightness_temperature_10"]
        tb_11 = kwargs["brightness_temperature_11"]
        emissivity_10 = kwargs["emissivity_10"]
        emissivity_11 = kwargs["emissivity_11"]
        mask = kwargs["mask"]

        mean_e = (emissivity_10 + emissivity_11) / 2
        diff_e = emissivity_10 - emissivity_11
        diff_tb = tb_10 - tb_11

        lst = (
            tb_10
            + (1.387 * diff_tb)
            + (0.183 * (diff_tb ** 2))
            - 0.268
            + ((54.3 - (2.238 * self.cwv)) * (1 - mean_e))
            + ((-129.2 + (16.4 * self.cwv)) * diff_e)
        )
        lst[mask] = np.nan
        return lst

In [21]:
emissivity = np.random.randn(50, 50)

In [22]:
emissivity.shape

(50, 50)

In [23]:
mask = emissivity < 0.5

In [26]:
if mask.dtype != 'bool':
    print('yaay')

In [20]:
lst = SplitWindowJiminezMunozLST()(
    emissivity_10=emissivity,
    emissivity_11=emissivity, 
    brightness_temperature_10=emissivity
    brightness_temperature_10=emissivity
    brightness_temperature_10=emissivity
)

KeywordArgumentError: Keyword arg brightness_temperature_10 should be provided

##### Visualize the  Land Surface Temperature obtained

In [ ]:
plt.imshow(lst_image_split_window)
plt.colorbar()
plt.title('{}\n LST in Celcius {}'.format(f'{method} method', lst_image_split_window.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')
plt.show()